 # Introductory Tutorial: Boltzmann Wealth Model with mesa-frames 💰🚀

In this tutorial, we'll implement the Boltzmann Wealth Model using mesa-frames. This model simulates the distribution of wealth among agents, where agents randomly give money to each other.

## Setting Up the Model 🏗️

First, let's import the necessary modules and set up our model class:


In [ ]:
from attr.validators import instance_of
from numba import typeof

from mesa_frames import ModelDF, AgentSetPandas, AgentSetPolars

class MoneyModelDF(ModelDF):
    def __init__(self, N: int, agents_cls):
        super().__init__()
        self.n_agents = N
        self.agents += agents_cls(N, self)

    def step(self):
        # Executes the step method for every agentset in self.agents
        self.agents.do("step")

    def run_model(self, n):
        for _ in range(n):
            self.step()


This `MoneyModelDF` class will work for both pandas and Polars implementations.

## Implementing the AgentSet 👥

Now, let's implement our `MoneyAgentSet` using both pandas and Polars backends. You can switch between the two implementations:

=== "pandas 🐼"


In [ ]:
        import pandas as pd
        import numpy as np

        class MoneyAgentPandas(AgentSetPandas):
            def __init__(self, n: int, model: ModelDF) -> None:
                super().__init__(model)
                self += pd.DataFrame(
                    {"unique_id": np.arange(n, dtype="int64"), "wealth": np.ones(n)}
                )

            def step(self) -> None:
                self.do("give_money")

            def give_money(self):
                self.select(self.wealth > 0)
                other_agents = self.agents.sample(n=len(self.active_agents), replace=True)
                self["active", "wealth"] -= 1
                new_wealth = other_agents.groupby("unique_id").count()
                self[new_wealth, "wealth"] += new_wealth["wealth"]


=== "Polars 🐻‍❄️"


In [ ]:
        import polars as pl

        class MoneyAgentPolars(AgentSetPolars):
            def __init__(self, n: int, model: ModelDF):
                super().__init__(model)
                self += pl.DataFrame(
                    {"unique_id": pl.arange(n, eager=True), "wealth": pl.ones(n, eager=True)}
                )

            def step(self) -> None:
                self.do("give_money")

            def give_money(self):
                self.select(self.wealth > 0)
                other_agents = self.agents.sample(n=len(self.active_agents), with_replacement=True)
                self["active", "wealth"] -= 1
                new_wealth = other_agents.group_by("unique_id").len()
                self[new_wealth["unique_id"], "wealth"] += new_wealth["len"]


## Running the Model ▶️

Now that we have our model and agent set defined, let's run a simulation:


In [ ]:
# Choose either MoneyAgentPandas or MoneyAgentPolars
agent_class = MoneyAgentPandas  # or MoneyAgentPolars

# Create and run the model
model = MoneyModelDF(1000, agent_class)
model.run_model(100)

wealth_dist=pd.DataFrame(model.agents)
# Print the final wealth distribution
print(wealth_dist["wealth"].describe())


Output:


```python
count    1000.000000
mean        1.000000
std         1.414214
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max        13.000000
Name: wealth, dtype: float64
```



This output shows the statistical summary of the wealth distribution after 100 steps of the simulation with 1000 agents.

## Performance Comparison 🏎️💨

One of the key advantages of mesa-frames is its performance with large numbers of agents. Let's compare the performance of our pandas and Polars implementations:


Add Polars and Pandas Concise/Native classes to test Mesa-Frames performance

In [ ]:

class MoneyAgentPolarsConcise(AgentSetPolars):
    def __init__(self, n: int, model: ModelDF):
        super().__init__(model)
        ## Adding the agents to the agent set
        # 1. Changing the agents attribute directly (not recommended, if other agents were added before, they will be lost)
        """self.agents = pl.DataFrame(
            {"unique_id": pl.arange(n, eager=True), "wealth": pl.ones(n, eager=True)}
        )"""
        # 2. Adding the dataframe with add
        """self.add(
            pl.DataFrame(
                {
                    "unique_id": pl.arange(n, eager=True),
                    "wealth": pl.ones(n, eager=True),
                }
            )
        )"""
        # 3. Adding the dataframe with __iadd__
        self += pl.DataFrame(
            {"unique_id": pl.arange(n, eager=True), "wealth": pl.ones(n, eager=True)}
        )

    def step(self) -> None:
        # The give_money method is called
        # self.give_money()
        self.do("give_money")

    def give_money(self):
        ## Active agents are changed to wealthy agents
        # 1. Using the __getitem__ method
        # self.select(self["wealth"] > 0)
        # 2. Using the fallback __getattr__ method
        self.select(self.wealth > 0)

        # Receiving agents are sampled (only native expressions currently supported)
        other_agents = self.agents.sample(
            n=len(self.active_agents), with_replacement=True
        )

        # Wealth of wealthy is decreased by 1
        # 1. Using the __setitem__ method with self.active_agents mask
        # self[self.active_agents, "wealth"] -= 1
        # 2. Using the __setitem__ method with "active" mask
        self["active", "wealth"] -= 1

        # Compute the income of the other agents (only native expressions currently supported)
        new_wealth = other_agents.group_by("unique_id").len()

        # Add the income to the other agents
        # 1. Using the set method
        """self.set(
            attr_names="wealth",
            values=pl.col("wealth") + new_wealth["len"],
            mask=new_wealth,
        )"""

        # 2. Using the __setitem__ method
        self[new_wealth, "wealth"] += new_wealth["len"]


class MoneyAgentPolarsNative(AgentSetPolars):
    def __init__(self, n: int, model: ModelDF):
        super().__init__(model)
        self += pl.DataFrame(
            {"unique_id": pl.arange(n, eager=True), "wealth": pl.ones(n, eager=True)}
        )

    def step(self) -> None:
        self.do("give_money")

    def give_money(self):
        ## Active agents are changed to wealthy agents
        self.select(pl.col("wealth") > 0)

        other_agents = self.agents.sample(
            n=len(self.active_agents), with_replacement=True
        )

        # Wealth of wealthy is decreased by 1
        self.agents = self.agents.with_columns(
            wealth=pl.when(pl.col("unique_id").is_in(self.active_agents["unique_id"]))
            .then(pl.col("wealth") - 1)
            .otherwise(pl.col("wealth"))
        )

        new_wealth = other_agents.group_by("unique_id").len()

        # Add the income to the other agents
        self.agents = (
            self.agents.join(new_wealth, on="unique_id", how="left")
            .fill_null(0)
            .with_columns(wealth=pl.col("wealth") + pl.col("len"))
            .drop("len")
        )


class MoneyAgentPandasConcise(AgentSetPandas):
    def __init__(self, n: int, model: ModelDF) -> None:
        super().__init__(model)
        ## Adding the agents to the agent set
        # 1. Changing the agents attribute directly (not recommended, if other agents were added before, they will be lost)
        # self.agents = pd.DataFrame({"unique_id": np.arange(n), "wealth": np.ones(n)})
        # 2. Adding the dataframe with add
        # self.add(pd.DataFrame({"unique_id": np.arange(n), "wealth": np.ones(n)}))
        # 3. Adding the dataframe with __iadd__
        self += pd.DataFrame(
            {"unique_id": np.arange(n, dtype="int64"), "wealth": np.ones(n)}
        )

    def step(self) -> None:
        # The give_money method is called
        self.do("give_money")

    def give_money(self):
        ## Active agents are changed to wealthy agents
        # 1. Using the __getitem__ method
        # self.select(self["wealth"] > 0)
        # 2. Using the fallback __getattr__ method
        self.select(self.wealth > 0)

        # Receiving agents are sampled (only native expressions currently supported)
        other_agents = self.agents.sample(n=len(self.active_agents), replace=True)

        # Wealth of wealthy is decreased by 1
        # 1. Using the __setitem__ method with self.active_agents mask
        # self[self.active_agents, "wealth"] -= 1
        # 2. Using the __setitem__ method with "active" mask
        self["active", "wealth"] -= 1

        # Compute the income of the other agents (only native expressions currently supported)
        new_wealth = other_agents.groupby("unique_id").count()

        # Add the income to the other agents
        # 1. Using the set method
        # self.set(attr_names="wealth", values=self["wealth"] + new_wealth["wealth"], mask=new_wealth)
        # 2. Using the __setitem__ method
        self[new_wealth, "wealth"] += new_wealth["wealth"]


class MoneyAgentPandasNative(AgentSetPandas):
    def __init__(self, n: int, model: ModelDF) -> None:
        super().__init__(model)
        ## Adding the agents to the agent set
        self += pd.DataFrame(
            {"unique_id": np.arange(n, dtype="int64"), "wealth": np.ones(n)}
        )

    def step(self) -> None:
        # The give_money method is called
        self.do("give_money")

    def give_money(self):
        self.select(self.agents["wealth"] > 0)

        # Receiving agents are sampled (only native expressions currently supported)
        other_agents = self.agents.sample(n=len(self.active_agents), replace=True)

        # Wealth of wealthy is decreased by 1
        b_mask = self.active_agents.index.isin(self.agents)
        self.agents.loc[b_mask, "wealth"] -= 1

        # Compute the income of the other agents (only native expressions currently supported)
        new_wealth = other_agents.groupby("unique_id").count()

        # Add the income to the other agents
        merged = pd.merge(
            self.agents, new_wealth, on="unique_id", how="left", suffixes=("", "_new")
        )
        merged["wealth"] = merged["wealth"] + merged["wealth_new"].fillna(0)
        self.agents = merged.drop(columns=["wealth_new"])



Add Mesa implementation of MoneyAgent and MoneyModel classes to test Mesa performance

In [ ]:
import mesa
import importlib.metadata
from packaging import version

class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, model):
        # Pass the parameters to the parent class.
        super().__init__(model)

        # Create the agent's variable and set the initial values.
        self.wealth = 1

    def step(self):
        # Verify agent has some wealth
        if self.wealth > 0:
            other_agent = self.random.choice(self.model.agents)
            if other_agent is not None:
                other_agent.wealth += 1
                self.wealth -= 1


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N):
        super().__init__()
        self.num_agents = N
        # Create scheduler and assign it to the model if version < 2.4.0
        installed_version = version.parse(importlib.metadata.version("mesa"))
        required_version = version.parse("2.4.0")

        if installed_version < required_version:
            self.agents = [MoneyAgent(self) for i in range(self.num_agents)]

    def step(self):
        """Advance the model by one step."""
        self.random.shuffle(self.agents)
        for agent in self.agents:
            agent.step()

    def run_model(self, n_steps) -> None:
        for _ in range(n_steps):
            self.step()

In [ ]:
import time

def run_mesa_simulation(model_class,  n_steps):
    start_time = time.time()
    model = model_class(n_steps)
    #     model.run_model(self=model)

    model.run_model(10)
    end_time = time.time()
    return end_time - start_time

def run_simulation(model_class, n_agents, n_steps):
    start_time = time.time()
    model = model_class(n_agents)
    #     model.run_model(self=model)

    model.run_model(n_agents)
    end_time = time.time()
    return end_time - start_time

# Compare mesa and mesa-frames implementations
n_agents_list = [100000, 300000, 500000, 700000]
n_steps = 100
a=1
print("Execution times:")
for implementation in ["mesa", "mesa-frames (pl concise)", "mesa-frames (pl native)", "mesa-frames (pd concise)", "mesa-frames (pd native)"]:
    print(f"---------------\n{implementation}:")
    for n_agents in n_agents_list:
        if implementation == "mesa":
            ntime = run_mesa_simulation(lambda n: MoneyModel(n), n_steps)
        if implementation == "mesa-frames (pl concise)":
            ntime = run_simulation(lambda n: MoneyModelDF(n, MoneyAgentPolarsConcise), n_agents, n_steps)
        elif implementation == "mesa-frames (pl native)":
            ntime = run_simulation(lambda n: MoneyModelDF(n, MoneyAgentPolarsNative), n_agents, n_steps)
        elif implementation == "mesa-frames (pd concise)":
            ntime = run_simulation(lambda n: MoneyModelDF(n, MoneyAgentPandasConcise), n_agents, n_steps)
        else:  # mesa-frames (pd native)
            ntime = run_simulation(lambda n: MoneyModelDF(n, MoneyAgentPandasNative), n_agents, n_steps)

        print(f"  Number of agents: {n_agents}, Time: {ntime:.2f} seconds")
    print("---------------")


Example output:


```plaintext
---------------
mesa:
  Number of agents: 100000, Time: 3.80 seconds
  Number of agents: 300000, Time: 14.96 seconds
  Number of agents: 500000, Time: 26.88 seconds
  Number of agents: 700000, Time: 40.34 seconds
---------------
---------------
mesa-frames (pl concise):
  Number of agents: 100000, Time: 0.76 seconds
  Number of agents: 300000, Time: 2.01 seconds
  Number of agents: 500000, Time: 4.77 seconds
  Number of agents: 700000, Time: 7.26 seconds
---------------
---------------
mesa-frames (pl native):
  Number of agents: 100000, Time: 0.35 seconds
  Number of agents: 300000, Time: 0.85 seconds
  Number of agents: 500000, Time: 1.55 seconds
  Number of agents: 700000, Time: 2.61 seconds
---------------
---------------
mesa-frames (pd concise):
  Number of agents: 100000, Time: 2.37 seconds
  Number of agents: 300000, Time: 7.47 seconds
  Number of agents: 500000, Time: 13.29 seconds
  Number of agents: 700000, Time: 18.32 seconds
---------------
---------------
mesa-frames (pd native):
  Number of agents: 100000, Time: 1.63 seconds
  Number of agents: 300000, Time: 5.76 seconds
  Number of agents: 500000, Time: 9.48 seconds
  Number of agents: 700000, Time: 13.58 seconds
---------------
```


Speed-up over mesa: 🚀


```plaintext
mesa-frames (pl concise):
  Number of agents: 100000, Speed-up: 5.00x 💨
  Number of agents: 300000, Speed-up: 7.44x 💨
  Number of agents: 500000, Speed-up: 5.63x 💨
  Number of agents: 700000, Speed-up: 5.56x 💨
---------------
mesa-frames (pl native):
  Number of agents: 100000, Speed-up: 10.86x 💨
  Number of agents: 300000, Speed-up: 17.60x 💨
  Number of agents: 500000, Speed-up: 17.34x 💨
  Number of agents: 700000, Speed-up: 15.46x 💨
---------------
mesa-frames (pd concise):
  Number of agents: 100000, Speed-up: 1.60x 💨
  Number of agents: 300000, Speed-up: 2.00x 💨
  Number of agents: 500000, Speed-up: 2.02x 💨
  Number of agents: 700000, Speed-up: 2.20x 💨
---------------
mesa-frames (pd native):
  Number of agents: 100000, Speed-up: 2.33x 💨
  Number of agents: 300000, Speed-up: 2.60x 💨
  Number of agents: 500000, Speed-up: 2.83x 💨
  Number of agents: 700000, Speed-up: 2.97x 💨
---------------
```


## Conclusion 🎉

- All mesa-frames implementations significantly outperform the original mesa implementation. 🏆
- The Polars backend consistently provides better performance than the pandas backend. 🐻‍❄️ > 🐼
- The native implementation for both Polars and pandas shows better performance than their concise counterparts. 💪
- The Polars native implementation shows the most impressive speed-up, ranging from 10.86x to 17.60x faster than mesa! 🚀🚀🚀
- Even the "slowest" mesa-frames implementation (pandas concise) is still 1.60x to 2.20x faster than mesa. 👍
- The performance advantage of mesa-frames becomes more pronounced as the number of agents increases. 📈
